<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Анализ-пользователей-ленты-новостей" data-toc-modified-id="Анализ-пользователей-ленты-новостей-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Анализ пользователей ленты новостей</a></span><ul class="toc-item"><li><span><a href="#Аудиторные-данные" data-toc-modified-id="Аудиторные-данные-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Аудиторные данные</a></span><ul class="toc-item"><li><span><a href="#DAU" data-toc-modified-id="DAU-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>DAU</a></span></li></ul></li><li><span><a href="#Характер-Retention-пользователей" data-toc-modified-id="Характер-Retention-пользователей-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Характер Retention пользователей</a></span><ul class="toc-item"><li><span><a href="#Активные-пользователи,-привлеченные-рекламной-компанией" data-toc-modified-id="Активные-пользователи,-привлеченные-рекламной-компанией-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Активные пользователи, привлеченные рекламной компанией</a></span></li><li><span><a href="#Активные-органические-пользователи" data-toc-modified-id="Активные-органические-пользователи-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Активные органические пользователи</a></span></li></ul></li><li><span><a href="#Пользователи,-которые-не-смогли-воспользоваться-лентой" data-toc-modified-id="Пользователи,-которые-не-смогли-воспользоваться-лентой-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Пользователи, которые не смогли воспользоваться лентой</a></span><ul class="toc-item"><li><span><a href="#Распределение-пользователей-по-странам" data-toc-modified-id="Распределение-пользователей-по-странам-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Распределение пользователей по странам</a></span></li><li><span><a href="#Распределение-пользователей-по-городам" data-toc-modified-id="Распределение-пользователей-по-городам-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Распределение пользователей по городам</a></span></li></ul></li></ul></li><li><span><a href="#Активность-аудитории" data-toc-modified-id="Активность-аудитории-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Активность аудитории</a></span><ul class="toc-item"><li><span><a href="#Аудитория-по-неделям-в-разрезе-пользователей" data-toc-modified-id="Аудитория-по-неделям-в-разрезе-пользователей-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Аудитория по неделям в разрезе пользователей</a></span></li><li><span><a href="#Активная-аудитория-по-неделям" data-toc-modified-id="Активная-аудитория-по-неделям-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Активная аудитория по неделям</a></span></li></ul></li></ul></div>

# Анализ продуктовых метрик

## Анализ пользователей ленты новостей

### Аудиторные данные

#### DAU

In [ ]:
SELECT toStartOfDay(toDateTime(time)) AS __timestamp,
       count(DISTINCT user_id) AS "Уникальные пользователи"
FROM feed_actions
GROUP BY toStartOfDay(toDateTime(time))
ORDER BY "Уникальные пользователи" DESC
LIMIT 50000;

### Характер Retention пользователей

#### Активные пользователи, привлеченные рекламной компанией

In [ ]:
SELECT toStartOfDay(toDateTime(date)) AS __timestamp,
       count(user_id) AS "Активные пользователи"
FROM
  (SELECT date, user_id,
                source
   FROM
     (SELECT user_id
      FROM feed_actions
      GROUP BY user_id
      HAVING min(toDate(time)) = '2024-10-10') t1
   JOIN
     (SELECT DISTINCT user_id,
                      toDate(time) AS date,
                      source
      FROM feed_actions) t2 USING user_id) AS virtual_table
WHERE ((source = 'ads'))
GROUP BY toStartOfDay(toDateTime(date))
ORDER BY "Активные пользователи" DESC
LIMIT 1000;

#### Активные органические пользователи

In [ ]:
SELECT toStartOfDay(toDateTime(date)) AS __timestamp,
       count(user_id) AS "Активные пользователи"
FROM
  (SELECT date, user_id,
                source
   FROM
     (SELECT user_id
      FROM feed_actions
      GROUP BY user_id
      HAVING min(toDate(time)) = '2024-10-10') t1
   JOIN
     (SELECT DISTINCT user_id,
                      toDate(time) AS date,
                      source
      FROM feed_actions) t2 USING user_id) AS virtual_table
WHERE ((source = 'organic'))
GROUP BY toStartOfDay(toDateTime(date))
ORDER BY "Активные пользователи" DESC
LIMIT 1000;

### Пользователи, которые не смогли воспользоваться лентой

#### Распределение пользователей по странам

In [ ]:
SELECT toStartOfDay(toDateTime(time)) AS __timestamp,
       country AS country,
       count(DISTINCT user_id) AS "COUNT_DISTINCT(user_id)"
FROM
  (SELECT *
   FROM feed_actions
   WHERE toDate(time) >= '2024-10-17'
     AND toDate(time) <= '2024-10-21') AS virtual_table
GROUP BY country,
         toStartOfDay(toDateTime(time))
ORDER BY "COUNT_DISTINCT(user_id)" DESC
LIMIT 1000;

#### Распределение пользователей по городам

In [ ]:
SELECT toStartOfDay(toDateTime(time)) AS __timestamp,
       city AS city,
       count(DISTINCT user_id) AS "COUNT_DISTINCT(user_id)"
FROM
  (SELECT *
   FROM feed_actions
   WHERE toDate(time) >= '2024-10-17'
     AND toDate(time) <= '2024-10-21') AS virtual_table
WHERE city IN ('Novosibirsk',
               'Yekaterinburg',
               'Moscow',
               'Saint Petersburg',
               'Chelyabinsk',
               'Nizhniy Novgorod')
GROUP BY city,
         toStartOfDay(toDateTime(time))
ORDER BY "COUNT_DISTINCT(user_id)" DESC
LIMIT 1000;

## Активность аудитории

### Аудитория по неделям в разрезе пользователей

In [ ]:
SELECT toStartOfDay(toDateTime(this_week)) AS __timestamp,
       status AS status,
       AVG(count_users) AS "AVG(count_users)"
FROM
  (SELECT this_week, -COUNT(DISTINCT user_id) AS count_users,
                      status
   FROM
     (SELECT user_id,
             groupUniqArray(toMonday(toDate(time))) AS week_visited,
             addWeeks(arrayJoin(week_visited), +1) AS this_week,
             addWeeks(this_week, -1) AS previous_week,
             if(has(week_visited, this_week) = 0, 'gone', 'retained') AS status
      FROM feed_actions
      GROUP BY user_id)
   WHERE status = 'gone'
   GROUP BY this_week,
            status
   UNION ALL SELECT this_week,
                    toInt64(COUNT(DISTINCT user_id)) AS count_users,
                    status
   FROM
     (SELECT user_id,
             groupUniqArray(toMonday(toDate(time))) AS week_visited,
             arrayJoin(week_visited) AS this_week,
             addWeeks(this_week, -1) AS previous_week,
             if(has(week_visited, addWeeks(this_week, -1)) = 0, 'new', 'retained') AS status
      FROM feed_actions
      GROUP BY user_id)
   GROUP BY this_week,
            status) AS virtual_table
GROUP BY status,
         toStartOfDay(toDateTime(this_week))
ORDER BY "AVG(count_users)" DESC
LIMIT 1000;

### Активная аудитория по неделям

In [ ]:
SELECT toMonday(toDateTime(time)) AS __timestamp,
       count(DISTINCT user_id) AS "Уникальные пользователи"
FROM feed_actions
GROUP BY toMonday(toDateTime(time))
ORDER BY "Уникальные пользователи" DESC
LIMIT 10000;